Notebook version 1.0, 31 Aug 2021. Written by Joona Andersson / CSC - IT Center for Science Ltd. joona.andersson77@gmail.com

Licensed under the MIT license: http://www.opensource.org/licenses/mit-license.php

Tested on Kvasi, running QLM version 1.2.1: https://research.csc.fi/-/kvasi
***
# QAOA for MaxCut, Circuit Sizes
Quantum Approximate Optimization Algorithms (QAOA) are possibly the first type of quantum algorithms to show quantum advantage in the near future. One application is the MaxCut algorithm. In this notebook we explore the sizes of QAOA circuits and how they grow as the size of the MaxCut problem graphs get larger.<br>

In [ ]:
## Import the necessary tools
import networkx as nx # this is a Python package for network/graph tools
import numpy as np
from qat.core.util import statistics
from qat.vsolve.qaoa import MaxCut
import matplotlib.pyplot as plt

In [ ]:
## Define some helper functions
# Generate two circuits of the same problem graph
def two_circuits(nodes, depth):
    graph = nx.generators.random_graphs.erdos_renyi_graph(n=nodes, p = 0.5)
    problem = MaxCut(graph)
    circuit1 = problem.qaoa_ansatz(depth=depth).circuit
    circuit2 = problem.qaoa_ansatz(depth=depth+1).circuit
    return circuit1, circuit2

def get_circuit(nodes, depth):
    graph = nx.generators.random_graphs.erdos_renyi_graph(n=nodes, p = 0.5)
    problem = MaxCut(graph)
    circuit = problem.qaoa_ansatz(depth = depth).circuit
    return circuit

## Investigate QAOA circuit sizes at a fixed depth
def get_number_of_cnots(fixed_depth, nodes_limit):
    number_of_cnots = dict()
    temp = []
    for nodes in range(5, nodes_limit):
        for _ in range(100):
            circuit = get_circuit(nodes, fixed_depth)
            temp.append(statistics(circuit)['gates']['CNOT'])
        number_of_cnots[nodes] = sum(temp) / 100
    return number_of_cnots

def get_total_number_of_gates(fixed_depth, nodes_limit):
    number_of_gates = {}
    temp = []
    for nodes in range(5, nodes_limit):
        for _ in range(100):
            circuit = get_circuit(nodes, fixed_depth)
            stats = statistics(circuit)
            temp.append(stats['gate_size'])
        number_of_gates[nodes] = sum(temp) / 100
    return number_of_gates

In [ ]:
## Visualize QAOA circuit for MaxCut
circuit_1, circuit_2 = two_circuits(nodes=4, depth=1)
print('One QAOA step / depth 1: ')
%qatdisplay circuit_1
print(statistics(circuit_1))
print('Two QAOA steps / depth 2: ')
%qatdisplay circuit_2
print(statistics(circuit_2))

In [ ]:
## Get the number of CNOT operations at different depths for plotting.
cnots_at_depth1 = get_number_of_cnots(1, 21)
cnots_at_depth2 = get_number_of_cnots(2, 21)
cnots_at_depth4 = get_number_of_cnots(4, 21)
cnots_at_depth8 = get_number_of_cnots(8, 21)

In [ ]:
plt.figure(figsize=(10, 5))
plt.rcParams['font.size'] = 16
plt.plot(cnots_at_depth1.keys(), cnots_at_depth1.values(), label='CNOTs at depth 1')
plt.plot(cnots_at_depth2.keys(), cnots_at_depth2.values(), label='CNOTs at depth 2')
plt.plot(cnots_at_depth4.keys(), cnots_at_depth4.values(), label='CNOTs at depth 4')
plt.plot(cnots_at_depth8.keys(), cnots_at_depth8.values(), label='CNOTs at depth 8')
plt.xlabel('Number of nodes in graph/ qubits')
plt.ylabel('Number of CNOTs in the circuit')
plt.legend()
plt.grid()
plt.show()

In [ ]:
## Get the total number of gates at different depths.
gates_at_depth1 = get_total_number_of_gates(1, 21)
gates_at_depth2 = get_total_number_of_gates(2, 21)
gates_at_depth4 = get_total_number_of_gates(4, 21)
gates_at_depth8 = get_total_number_of_gates(8, 21)

In [ ]:
plt.figure(figsize=(10, 5))
plt.rcParams['font.size'] = 16
plt.plot(gates_at_depth1.keys(), gates_at_depth1.values(), label='Gates at depth 1')
plt.plot(gates_at_depth2.keys(), gates_at_depth2.values(), label='Gates at depth 2')
plt.plot(gates_at_depth4.keys(), gates_at_depth4.values(), label='Gates at depth 4')
plt.plot(gates_at_depth8.keys(), gates_at_depth8.values(), label='Gates at depth 8')
plt.xlabel('Number of nodes in graph/ qubits')
plt.ylabel('Number of gates in the circuit')
plt.legend()
plt.grid()
plt.show()

## Requirements For Achieving Quantum Advantage With MaxCut
A study has found that in order for QAOA for MaxCut to be competitive against classical algorithms and to outperform them, we would need to use hundreds to a few thousands of qubits. Another study suggests that QAOA depth of at least 8 will be necessary to ensure good quality results for computations with larger graphs. <br>

Let's use the data generated by Kvasi to find out how large the circuits necessary for quantum advantage might be, assuming circuits keep growing at the same rate.

In [ ]:
gates = {}
with open('depth_8_gates.txt', 'r') as file:
    for line in file:
        line = line.strip().split(',')
        gates[int(line[0])] = float(line[1])

model = np.polyfit(list(gates.keys()), list(gates.values()), 2)
predict = np.poly1d(model)

x = np.arange(5, 700)
plt.figure(figsize=(15, 8))
plt.rcParams['font.size'] = 16
plt.plot(x, predict(x), label='Number of gates')
plt.title('Number of Quantum Gates at QAOA depth 8')
plt.xlabel('Number of graph nodes/qubits')
plt.ylabel('Number of quantum gates')
plt.legend()
plt.grid()
plt.show()

As we can see, the circuit for MaxCut gets very large as we reach graph sizes where quantum computation might outperform classical computing. <br>

Ideally, the number of qubits grows as the number of nodes in the problem graph grows. However, with larger circuits, the need to implement quantum error correction will arise. The plots above assume that all qubits are connected to each other which is not how quantum processors work in reality. In addition, PH gates might have to be decomposed further into many rotations to implement a specific angle for a rotation. Taking these three factors into account might significantly increase the number of qubits required and the number of gates in the circuits.
<br>
## Links
[QAOA for Max-Cut requires hundreds of qubits for quantum speed-up](https://arxiv.org/pdf/1812.07589.pdf) <br>
[Performance of the Quantum Approximate Optimization Algorithm on the Maximum Cut Problem](https://arxiv.org/pdf/1811.08419.pdf) <br>
[Optimal ancilla-free Clifford+T approximation of z-rotations](https://arxiv.org/pdf/1403.2975.pdf)
